In [10]:
# DFP算法：拟牛顿方法
import numpy as np
class DFP(object):
    def __init__(self,x0,g0):
        super().__init__()
        self.x0 = x0
        self.g0 = g0
        # 初始化G0为一个单位矩阵（正定矩阵）
        self.G0 = np.eye(len(x0))
        
    # 根据算法，使用下一轮的xk+1和gk+1更新矩阵Gk+1
    def update_quasi_newton_matrix(self,x1,g1):
        y0 = g1 - self.g0
        delta0 = x1 - self.x0
        self.G0 = self.G0 + delta0 @ delta0.T /( (delta0.T @ y0)[0,0] + 1e-7) - self.G0 @ y0 @ y0.T @ self.G0 / ((y0.T @ self.G0 @ y0)[0,0] + 1e-7)
    
    # 对原始的梯度做调整，变成牛顿方向
    # 也就是求得更新方向的反方向
    def adjust_gradient(self,gradient):
        return self.G0 @ gradient

In [25]:
# 使用拟牛顿法对凸函数就行最优值搜索

def func(x):
    return x[0,0]*x[0,0] + x[1,0]*x[1,0] + 1

def gradient(x):
    return np.array([[2 * x[0,0]],[2 * x[1,0]]])

x = np.array([[1],[1]])
g = gradient(x=x)
dfp = DFP(x,g)
for _ in range(1000):
    dx = gradient(x)
    dfp.update_quasi_newton_matrix(x,dx)
    x = x - 0.05 * dfp.adjust_gradient(dx)
    
x,func(x)

(array([[5.01330576e-23],
        [5.01330576e-23]]),
 1.0)

In [26]:
# BFGS算法
class BFGS(object):
    def __init__(self,x0,g0):
        super().__init__()
        self.x0 = x0
        self.g0 = g0
        # 初始化G0为一个单位矩阵（正定矩阵）
        self.B0 = np.eye(len(x0))
        
    # 根据算法，使用下一轮的xk+1和gk+1更新矩阵Gk+1
    def update_quasi_newton_matrix(self,x1,g1):
        y0 = g1 - self.g0
        delta0 = x1 - self.x0
        self.B0 = self.B0 + y0 @ y0.T  /( (y0.T @ delta0)[0,0] + 1e-7) - self.B0 @ delta0 @ delta0.T @ self.B0 / (( delta0.T @ self.B0 @ delta0 )[0,0] + 1e-7)
    
    # 对原始的梯度做调整，变成牛顿方向
    # 也就是求得更新方向的反方向
    def adjust_gradient(self,gradient):
        return np.linalg.pinv(self.B0) @ gradient

In [30]:
# 使用拟牛顿法对凸函数就行最优值搜索

def func(x):
    return x[0,0]*x[0,0] + x[1,0]*x[1,0] + 1

def gradient(x):
    return np.array([[2 * x[0,0]],[2 * x[1,0]]])

x = np.array([[1],[1]])
g = gradient(x=x)
# 使用BFGS方法对梯度进行更新
bfgs = BFGS(x,g)
for _ in range(100):
    dx = gradient(x)
    bfgs.update_quasi_newton_matrix(x,dx)
    x = x - 0.05 * bfgs.adjust_gradient(dx)
    
x,func(x)

(array([[0.00560892],
        [0.00560892]]),
 1.0000629200214106)